# QEC in one glance

*Want to follow along? {nb-download}`Download this notebook.<2. quick start.ipynb>`*

This notebook will demo you the essence of QEC experiment in a bird's eye view.

If you don't have this notebook on your drive, please download it and put in the same folder you are already using for venv and jupyter. Done? Let's go!

## Experiment in one cell

QEC experiment today is build around performing some error-corrected operation multiple times. One of the types of such operation is called **quantum memory experiment**.
This experiment prepares a logical basis state (e.g. $|0\rangle_L$), performs error correction of a logical qubit for some number of rounds and then measures the logical state ($|\psi\rangle_L$, which might be the same, of flipped).
Measured states in multiple "shots" are compared with predictions of a decoder. The closer predictions are to the logical state - the better is the combination of the code and the decoder.

Quantum Z-memory experiment (this means that preparation and measurements is done in Z computation basis) may be expressed as logical quantum pseudocode:

```
reset qubit0;
id qubit0;
measure qubit0;
```

In an error corrected circuit `qubit0` becomes a set of qubits (aka data qubits). Unlike in classical error correction, these qubits
cannot be measured while executing the algorithm, without disrupting the calculations. That is why data qubits are sneakily followed by ancillary qubits.
Ancillary qubits entagle with neighbouring data qubits and perform the role of parity check. Every round of an experiment we measure
ancillary qubits to learn if a state flip happened to their neighbours.

QEC codes define which qubits perform which roles and how they entangle. Decoders use ancillary qubit measurements to predict corruption of a logical state.

In [ ]:
from deltakit.circuit.gates import PauliBasis
from deltakit.explorer import enums, types

experiment = types.QECExperimentDefinition(
    experiment_type=enums.QECExperimentType.QUANTUM_MEMORY,  # we perform a quantum memory experiment
    code_type=enums.QECECodeType.REPETITION,                 # we use repetition code for QEC
    observable_basis=PauliBasis.Z,                           # this is a Z-basis experiment. We reset and measure in Z
    num_rounds=1,                                            # smallest number of "waiting rounds"
    parameters=types.CircuitParameters.from_sizes([3]),      # this is distance-3 (can correct chains of errors up to 3)
    # ion trap gates
    basis_gates = ["SQRT_XX", "X", "Y", "Z", "SQRT_X", "SQRT_X_DAG", "S", "S_DAG", "MZ", "RZ"],
)

This is all we need to define a circuit for an experiment. Here are the following steps:

1. Generate a circuit for the experiment.
2. Simulate this circuit a statistically significant number of times.
3. Decode simulation results.
4. Perform required analysis to answer your research question.

In [ ]:
from __future__ import annotations
from deltakit.explorer import Client, enums, types


client = Client.get_instance()

# 1. get noisy circuit
circuit = client.generate_circuit(experiment)
noise = types.PhysicalNoiseModel.get_ion_trap_noise()
noisy_circuit = client.add_noise(circuit, noise)

# 2. Simulate and get QEC-specific data
measurements, _ = client.simulate_stim_circuit(noisy_circuit, 500_000)
detectors, observables = measurements.to_detectors_and_observables(noisy_circuit)

# 3. decode
decoder1 = types.Decoder(enums.DecoderType.MWPM)
decoding_result1 = client.decode(detectors, observables, decoder1, noisy_circuit)

decoder2 = types.Decoder(enums.DecoderType.AC)
decoding_result2 = client.decode(detectors, observables, decoder2, noisy_circuit)

# 4. analyse

def print_result(decoder: str, result: types.DecodingResult):
    print(
        f"Decoder {decoder}: "
        f"LEP={result.get_logical_error_probability():.5f}"
        f"±{result.get_standard_deviation():.5f} "
        f"TIME={sum(result.times):.5f}s"
    )

print_result("MWPM", decoding_result1)
print_result("AC", decoding_result2)

## Exercise #1

We've decoded **one** particular experiment, and every time you run it, you should have similar numbers.
What we are often interested in is HOW the logical error probability responds to different parameter tweaking.

Your first task is to complete the following code. For a fixed noise model and a fixed decoder,
plot how the repetition code LEP scales with size.

In [ ]:
import matplotlib.pyplot as plt
import tqdm

distances = [3, 5, 7, 9, 11]
le_probabilities = []
for distance in tqdm.tqdm(distances):
    # update experiment parameters
    experiment.parameters = types.CircuitParameters.from_sizes([distance])
    experiment.num_rounds = distance
    # define noise
    noise = types.SI1000NoiseModel(0.01, 0.0)

    # TODO: your code here!
    # Add in the steps required to generate circuit,
    # add noise, simulate and decode the syndrome measurements.
    decoding_result = ...
    
    le_probabilities.append(decoding_result.get_logical_error_probability())

plt.plot(distances, le_probabilities, label=f"repcode+{enums.DecoderType.MWPM.name}")

plt.xlabel("distance")
plt.ylabel("logical error probability")
plt.legend()
plt.grid()

## Exercise #2

Modify your code to compare `MWPM` and `BELIEF_MATCHING` decoders performance on the same simulation data.

## After all

Now you are ready to dive deeper into the details of experiment structure. Please, proceed to [Experiment step-by-step](3.%20experiment%20step%20by%20step.ipynb) notebook.